<a href="https://colab.research.google.com/github/geoskimoto/AWDB_SOAP_Request/blob/main/GetData_AWDB_SOAP_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library Imports

In [169]:
from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
# import statsmodels.api as sm
from functools import reduce

import requests
import xml.dom.minidom as minidom
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import datetime


# Helper Functions

In [ ]:
# Web Call to Access and Download Data From AWDB Web Service (SOAP API)

def SOAP_Call(stationtriplets, elementCD, begindate, enddate):

  # Create a dictionaries to store the data
  headers = {'Content-type': 'text/soap'}
  # current_dictionary = {}
  
  # Define Web Service URL
  URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

  # Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
  SOAP_current = '''
  <?xml version="1.0" encoding="UTF-8"?>
  <SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <SOAP-ENV:Body>
      <q0:getData>
        <stationTriplets>STATIONTRIPLETS</stationTriplets>
        <elementCd>ELEMENTCD</elementCd>   
        <ordinal>1</ordinal>
        <duration>DAILY</duration>
        <getFlags>false</getFlags>
        <beginDate>BEGINDATE</beginDate>
        <endDate>ENDDATE</endDate>
        <alwaysReturnDailyFeb29>false</alwaysReturnDailyFeb29>   
      </q0:getData>
    </SOAP-ENV:Body>
  </SOAP-ENV:Envelope>

  '''.strip()
  #Read GetData documents - If <alwaysReturnDailyFeb29> is set to true, will set a null for every non leap year on the 29th,  
  #which breaks this request when selecting date ranges that include Feb 29.
  #other element codes: PREC, WTEQ (Water Equivalent/SWE)
  

  # Post SOAP Elements to AWDB Web Service and process results - getData
  SOAP_current = SOAP_current.replace("ELEMENTCD", elementCD)
  SOAP_current = SOAP_current.replace("STATIONTRIPLETS", stationtriplets)
  SOAP_current = SOAP_current.replace("BEGINDATE", begindate)
  SOAP_current = SOAP_current.replace("ENDDATE", enddate)

  response_current = requests.post(URL, data=SOAP_current, headers=headers)
  xmldoc = minidom.parseString(response_current.text)

  val_length = len(xmldoc.getElementsByTagName('values'))
  data = pd.DataFrame([xmldoc.getElementsByTagName('values')[i].firstChild.data for i in range(0,val_length)])

  date = datetime.datetime.strptime(begindate, "%m/%d/%Y").date()  #https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior; .date() after .strptime just tells it to make it a datetime object.  Def necessary.
  # print(date)
  Date = []                                                       
  for i in range(0, val_length): 
    date += datetime.timedelta(days=1)
    Date.append((date))

  # {str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{Date[j]:xmldoc.getElementsByTagName('values')[j].firstChild.data} for j in range(3)}

  data['Date'] = Date
  data.columns = [f'{elementCD}','Date']
  data.set_index('Date', inplace=True)
  
  data[f'{elementCD}'] = list(map(lambda x: float(x), data[f'{elementCD}']))
  
  return data
  # data.tail(7)

In [172]:
# Function to Download Multiple Stations from AWDB Web Service
stations = ['401:OR:SNTL', '471:ID:SNTL', '591:WA:SNTL']
def getData(stations, parameter_of_interest):
  data = []
  for i in stations:
    data.append(SOAP_Call(stationtriplets=i,elementCD=parameter_of_interest,begindate='01/01/1991',enddate='06/01/1992'))
  return data

# Regression with selected data

### Fit Model to Parameter and Date Range of Choice

In [173]:
def fitData(stations, parameter_of_interest, begindate = begindate, enddate=enddate, regression_model='Random Forest'):

#Choose Regression Model
  if regression_model == 'Lasso':
    regr0 = LassoCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
  elif regression_model == 'Ridge':
    regr0 = RidgeCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
  elif regression_model == 'SVM':
    regr0 = svm.SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
  elif regression_model == 'Huber':
    regr0 = HuberRegressor()
  elif regression_model == 'GradientBoost':
    regr0 = GradientBoostingRegressor(random_state=0, n_estimators=100)
  elif regression_model == 'AdaBoost':
    regr0 = AdaBoostRegressor(random_state=0, n_estimators=100)
  elif regression_model == 'Random Forest':
    regr0 = RandomForestRegressor(n_estimators=100)
  else:
    print('Choose either Lasso, Ridge, SVM, Huber, GradientBoost, AdaBoost, or Random Forest')

#Download Data from AWDB Web Service
  data = getData(stations, parameter_of_interest='WTEQ')

#Define Targets and Features (e.g. Dependent and Independent Variables)
  target0 = data[0][parameter_of_interest]   #Need to keep a copy of target in a dataFrame in order to extract index/datetime for plots
  target = target0.tolist()

  features0 = data[1:]
  #Need to do this shinanagins in order to place features into a list (instead of a df).  This is required in order for it to be reshaped to fit the appropriate format for sklearn regr0.fit() and regr0.predict()
  features_list = []
  for i in features0:
    i.reset_index(inplace=True)
    features_list.append(i[parameter_of_interest].tolist())

  features = np.array(features_list).reshape(-1,len(stations)-1)

#Fit Mode
  regr0.fit(features, target)

#Split into training and test sets
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33)



  y_pred = regr0.predict(features)
  # # y_pred2 = pd.DataFrame(y_pred, columns=['y_pred'])
  # # y_pred2['Date'] = target0.index
  # # y_pred2.set_index('Date', inplace=True)

  # # site_list['y_pred'] = y_pred2
  # # abs_error = abs(y_pred - target)
  # # abs_error = pd.DataFrame(abs_error)
  # # abs_error['Date'] = site_list.index
  # # abs_error.set_index('Date',inplace=True)
  # # site_list['abs_error'] = abs_error
  # # site_list_sorted = site_list.sort_values('abs_error',ascending=False)

  fig = make_subplots(
      rows=2, cols=1,
      subplot_titles=('Predictions vs. Actual','Regression Plot')) 


  # 'Predictions vs. Actual'
  fig.add_trace(go.Scatter(
      y = target0,
      x = target0.index,
      mode = 'lines',
      name = f'{stations[0]}'
  ), 
  row=1, col=1)

  fig.add_trace(go.Scatter(
      y = y_pred,
      x = target0.index,
      mode = 'lines',
      name = 'model predictions'
  ), 
  row=1, col=1)

  # # # Regression Plot # # # 
  fig.add_trace(go.Scatter(
    x=data[1][parameter_of_interest],
    y=target0,
    mode='markers',
    # hovertext = site_list.index,
    marker=dict(size=5, line=dict(width=1.2))   
  ), 
  row=2, col=1
  )

  fig.add_trace(go.Scatter(
    x=data[1][parameter_of_interest],
    y=y_pred,
    mode='lines',
    hovertext = y_pred,  
  ), 
  row=2, col=1)

  fig.update_xaxes(title_text = f"{parameter_of_interest}", row=1, col=1)
  fig.update_yaxes(title_text = 'Date', row=1, col=1) 

  # fig.update_xaxes(title_text = f"{site_list.columns[0]}", row=2, col=1)
  # fig.update_yaxes(title_text = f'{site_list.columns[1:len(stations)', row=2, col=1) 

  fig.update_layout(
      showlegend=False,
      height=1550,
      width=1150 
  )
  fig.show()

### Make Predictions with New Date Range Using Fitted Model

In [160]:
at

In [160]:
# stations = ['401:OR:SNTL', '471:ID:SNTL', '591:WA:SNTL']
stations = ['777:WA:SNTL', '748:WA:SNTL', '553:WA:SNTL']
parameter_of_interest = 'WTEQ'
begindate='01/01/1990'
enddate='06/01/1992'

In [174]:
# stations = ['401:OR:SNTL', '471:ID:SNTL', '591:WA:SNTL']
stations = ['777:WA:SNTL', '748:WA:SNTL']#, '553:WA:SNTL']
parameter_of_interest = 'WTEQ'
begindate='01/01/1990'
enddate='06/01/2002'

fitData(stations, parameter_of_interest, begindate = begindate, enddate=enddate, regression_model='Huber')